In [11]:
from analysis_2019_08 import *
import pyfftw

In [12]:
data_path = '/media/ntlhui/FA56-CFCD/'
run_paths = glob.glob(os.path.join(data_path, '**', 'META_*'), recursive=True)
runs = [RCTRun(os.path.dirname(run_path)) for run_path in run_paths]
print('Found %d runs' % (len(runs)))
processable_runs = [run for run in runs if len(run.getFreqs()) > 0]
print("Processing %s" % (processable_runs[201].getPath()))

Found 278 runs
Processing /media/ntlhui/FA56-CFCD/2019.08.09.Successful_Night_Tracking/RUN_000069


In [13]:
run_num = 201
dataFiles = sorted(processable_runs[run_num].getDataFiles())
assert(len(dataFiles) > 0)
assert(processable_runs[run_num].getCenterFreq() is not None)
assert(len(processable_runs[run_num].getFreqs()) > 0)
assert(processable_runs[run_num].getSamplingFreq() is not None)
assert(processable_runs[run_num].getPingWidth() is not None)
assert(processable_runs[run_num].getSamplingFreq() is not None)
assert(processable_runs[run_num].getMinSNR() is not None)
assert(processable_runs[run_num].getStartTime_ms() is not None)
assert(processable_runs[run_num].getPingFactor()[0] is not None)
assert(processable_runs[run_num].getPingFactor()[1] is not None)

fft_bins = ((np.array(processable_runs[run_num].getFreqs()) - processable_runs[run_num].getCenterFreq()) / (processable_runs[run_num].getSamplingFreq() / 2) * FFT_LEN).astype(np.int)
print(processable_runs[run_num].getFreqs())
print(fft_bins)

# frequencyStep is the number of SDR samples between FFT windows
frequencyStep = 256
# fftFrequency is the sample frequency of FFTs in Hz
fftFrequency = processable_runs[run_num].getSamplingFreq() / frequencyStep
print("FFT Frequency: %f Hz" % (fftFrequency))
# integrateStep is the number of FFT samples between integration windows
integrateStep = 5
# integrateFactor is number of FFT samples to integrate
integrateFactor = 5
# integrateFactor = int(6e-3 * processable_runs[1].getSamplingFreq() / FFT_LEN * frequencyStepFactor)
# classifierInputFrequency is the sample frequency of the input signal to the classifier stage
classifierInputFreq = fftFrequency / integrateStep
print("Classifier Input Frequency: %f Hz" % (classifierInputFreq))
# pingWidth_samp is the floor of the ping width in samples at the classifier sample frequency
pingWidth_samp = int(processable_runs[run_num].getPingWidth() / 1000. * classifierInputFreq)
print("Ping Width: %d samples" % (pingWidth_samp))
dataLen = pingWidth_samp * 2
print("Databuf Length: %d samples" % (dataLen))
maximizerLen = int(0.1 * classifierInputFreq)
medianLen = int(1 * classifierInputFreq)
idLen = int(0.5 * classifierInputFreq)
readLen = int(frequencyStep)
SAMPLE_SIZE = 4


integrateCounter = 0
idIdx = 0
sampIdx = 0
integrator = CircularBuffer(integrateFactor)
dataBuf = CircularBuffer(dataLen)
peakHistory = CircularBuffer(maximizerLen)
peaks = CircularBuffer(medianLen)
idSignal = CircularBuffer(idLen)
dataFile = open(dataFiles[0], 'rb')
minSNRVec = np.ones(fft_bins.shape) * processable_runs[run_num].getMinSNR()
fftBuffer = CircularBuffer(FFT_LEN)
for i in range(FFT_LEN):
    fftBuffer.add(np.array([0+0j]))
pings = []
outputSignal = []
thresholdSignal = []
integratorSignal = []

pyfftw.config.NUM_THREADS = 7

pyfftw.config.PLANNER_EFFORT = 'FFTW_MEASURE'
fft_in = pyfftw.empty_aligned(FFT_LEN, dtype='complex64')
fft_out = pyfftw.empty_aligned(FFT_LEN, dtype='complex64')

fft_object = pyfftw.FFTW(fft_in, fft_out, threads=7)

print("Rejecting pings narrower than %f or wider than %f" % (processable_runs[run_num].getPingFactor()[0] * processable_runs[run_num].getPingWidth(), processable_runs[run_num].getPingFactor()[1] * processable_runs[run_num].getPingWidth()))
print("Rejecting pings with amplitude %f less than noise" % (processable_runs[run_num].getMinSNR()))

print("FFT Frequency: %.3f Hz" % (fftFrequency))
print("Integration length: %d samples" % (integrateFactor))
print("Classifier Frequency: %.3f Hz" % (classifierInputFreq))
print("Ping width: %d samples" % (pingWidth_samp))

[172642000]
[387]
FFT Frequency: 5859.375000 Hz
Classifier Input Frequency: 1171.875000 Hz
Ping Width: 42 samples
Databuf Length: 84 samples
Rejecting pings narrower than 27.000000 or wider than 54.000000
Rejecting pings with amplitude 1.000000 less than noise
FFT Frequency: 5859.375 Hz
Integration length: 5 samples
Classifier Frequency: 1171.875 Hz
Ping width: 42 samples


In [ ]:
while True:
    data = dataFile.read(readLen * SAMPLE_SIZE)
    if len(data) != readLen * SAMPLE_SIZE:
        missing = readLen * SAMPLE_SIZE - len(data)
        dataFiles.pop(0)
        if len(dataFiles) == 0:
            break
        dataFile = open(dataFiles[0], 'rb')
        data += dataFile.read(missing)
    block = struct.unpack("hh" * int(readLen), data)
    [fftBuffer.add(np.array([(block[2 * i] + block[2 * i + 1] * 1j) / processable_runs[run_num].getDataScalar()])) for i in range(int(readLen))]
    sampIdx += FFT_LEN
    a = fftBuffer.to_array()
    for i in range(FFT_LEN):
        fft_in[i] = a[i]
        
    complexFFT = fft_object() / FFT_LEN
    selectFreqs = complexFFT[fft_bins]
    integrator.add(np.abs(selectFreqs) ** 2)
    integrateCounter += 1
    if integrateCounter < integrateStep:
        continue
    integratedVector = integrator.sum()
    if len(dataBuf) == 0:
        for i in range(medianLen):
            peaks.add(integratedVector)
        for i in range(pingWidth_samp):
            peakHistory.add(integratedVector)
        for i in range(dataLen):
            dataBuf.add(integratedVector)
    dataBuf.add(integratedVector)
    peakHistory.add(integratedVector)

    peaks.add(peakHistory.max())

    threshold = np.multiply(peaks.median(), minSNRVec)
    thresholdSignal.append(threshold)
    outputSignal.append(integratedVector)

    # compare(sig, thresh, minSNR)
    sigCompare = integratedVector > threshold
    idSignal.add(sigCompare)
    idIdx += 1

    edges = processable_runs[run_num]._hasFallingEdge(idSignal)
    if len(edges) > 0:
        for edge in list(edges):
            width = processable_runs[run_num]._getPulseWidth(idSignal, edge)
            if width is None:
                continue
            if width > len(dataBuf) - 1:
                width = len(dataBuf) - 1
            pingStart_ms = (idIdx - width) / classifierInputFreq * 1e3
            amplitude = processable_runs[run_num]._getPulseAmplitude(width, dataBuf, edge)
            if (width + 0.5) / classifierInputFreq * 1e3 < processable_runs[run_num].getPingFactor()[0] * processable_runs[run_num].getPingWidth() or \
                (width - 0.5) / classifierInputFreq * 1e3 > processable_runs[run_num].getPingFactor()[1] * processable_runs[run_num].getPingWidth():
                print("Ping %d at %.3f ms, amplitude: %.3f, threshold: %.3f, width: %.3f ms (%d samples), freq: %d Hz, idx: %d, bin: %d rejected for length" % 
                    (len(pings) + 1, pingStart_ms + processable_runs[1].getStartTime_ms(), amplitude, threshold[edge],
                        width / classifierInputFreq * 1e3, width, processable_runs[1].getFreqs()[edge], edge, fft_bins[edge]))
                continue
            pings.append(Ping(pingStart_ms + processable_runs[run_num].getStartTime_ms(), amplitude, processable_runs[run_num].getFreqs()[edge], idIdx, width))
            print("Ping %d at %.3f ms, amplitude: %.3f, threshold: %.3f, width: %.3f ms (%d samples), freq: %d Hz, idx: %d, bin: %d" % 
                (len(pings), pingStart_ms + processable_runs[run_num].getStartTime_ms(), amplitude, threshold[edge],
                    width / classifierInputFreq * 1e3, width, processable_runs[run_num].getFreqs()[edge], edge, fft_bins[edge]))

    integrateCounter = 0

Ping 1 at 1502770878.398 ms, amplitude: 0.000, threshold: 0.000, width: 4.267 ms (5 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 1 at 1502770883.518 ms, amplitude: 0.000, threshold: 0.000, width: 7.680 ms (9 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 1 at 1502770892.052 ms, amplitude: 0.000, threshold: 0.000, width: 5.120 ms (6 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 1 at 1502770898.025 ms, amplitude: 0.000, threshold: 0.000, width: 5.120 ms (6 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 1 at 1502770903.998 ms, amplitude: 0.000, threshold: 0.000, width: 3.413 ms (4 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 1 at 1502770910.825 ms, amplitude: 0.000, threshold: 0.000, width: 2.560 ms (3 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 1 at 1502770915.092 ms, amplitude: 0.000, threshold: 0.000, width: 2.560 ms (3 samples), 

Ping 2 at 1502771194.985 ms, amplitude: 0.000, threshold: 0.000, width: 3.413 ms (4 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502771200.958 ms, amplitude: 0.000, threshold: 0.000, width: 19.627 ms (23 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502771222.292 ms, amplitude: 0.000, threshold: 0.000, width: 1.707 ms (2 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502771224.852 ms, amplitude: 0.000, threshold: 0.000, width: 19.627 ms (23 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502771245.332 ms, amplitude: 0.000, threshold: 0.000, width: 1.707 ms (2 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502771248.745 ms, amplitude: 0.000, threshold: 0.000, width: 8.533 ms (10 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502771258.132 ms, amplitude: 0.000, threshold: 0.000, width: 14.507 ms (17 sam

Ping 2 at 1502775105.812 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502775343.892 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502775367.785 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502775385.705 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502775398.505 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502775700.585 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502775719.358 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), 

Ping 2 at 1502782277.225 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502782287.465 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502782296.852 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502782406.078 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502782470.078 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502782570.772 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502782719.252 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), 

Ping 2 at 1502790039.998 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502790117.652 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502790148.372 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502790156.905 ms, amplitude: 0.000, threshold: 0.000, width: 2.560 ms (3 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502790185.065 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502790194.452 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502790196.158 ms, amplitude: 0.000, threshold: 0.000, width: 3.413 ms (4 samples), 

...

Ping 2 at 1502805389.758 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502805681.598 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502805896.638 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502805971.732 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502806094.612 ms, amplitude: 0.000, threshold: 0.000, width: 1.707 ms (2 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502806105.705 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502806231.145 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), 

Ping 2 at 1502813266.025 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502813280.532 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502813314.665 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502813356.478 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502813363.305 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502813380.372 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502813487.892 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), 

Ping 2 at 1502819076.372 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502819177.918 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502819282.025 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502819307.625 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502819543.145 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502819985.172 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502820143.038 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), 

Ping 2 at 1502827044.798 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502827139.518 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502827662.612 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502827732.585 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502827741.118 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502827887.892 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502827895.572 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), 

Ping 2 at 1502834407.358 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502834626.665 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502834684.692 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502834818.665 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502835055.038 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502835301.652 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), freq: 172012000 Hz, idx: 0, bin: 387 rejected for length
Ping 2 at 1502835325.545 ms, amplitude: 0.000, threshold: 0.000, width: 0.853 ms (1 samples), 

In [ ]:
retval = (outputSignal, thresholdSignal, pings)
plt.plot(retval[0])
plt.plot(retval[1])

minY = min(np.min(retval[0]), np.min(retval[1]))
maxY = max(np.max(retval[0]), np.max(retval[1]))

for ping in retval[2]:
    ax = plt.gca()
    rect = patches.Rectangle((ping.idx - ping.width_s, minY), ping.width_s, maxY - minY, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)